In [1]:
import pickle
import nltk
from collections import defaultdict
import random
import numpy as np
from tqdm.notebook import tqdm
import re

In [2]:
import pandas as pd
import keras
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
import os
from google.colab import drive

drive.mount('/content/gdrive')
print(os.getcwd())

Mounted at /content/gdrive
/content


In [4]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Dataset")
os.getcwd() 

'/content/gdrive/My Drive/Colab Notebooks/Dataset'

In [5]:
with open("Pre Data/Movies.txt", "r", encoding = "utf-8") as f:
    data = f.read()
    
words = data.split()    
print("Number of words:", len(words))

Number of words: 16264927


In [6]:
data[0:200]

'i admit the great majority of films released before say are just not for me. of the dozen or so major silents i have viewed one i loved the crowd and two we are very good the last command and city lig'

In [7]:
fp = data[300000:1650000].split()
word_count = len(fp)
print(word_count)

pre_data = data[300000:1650000]

250167


In [8]:
test_data = data[1650000:1850000]

In [9]:
print(type(pre_data))
pre_data[:22]

<class 'str'>


'scare me final destina'

In [10]:
def remove_dot(text):
  text = re.sub(r'[^\w\s]','',text)
  r = re.split("[^a-zA-Z\d]+",text)
  ans = ' '.join([ i for i in r if len(i) > 0 ])
  return ans

pre_data = remove_dot(pre_data)

In [11]:
test_data = remove_dot(test_data)

In [12]:
pre_data[:200]

'scare me final destination and did not scare me but the drag scenes made me squirm in my seatrating watching this movie after seeing the french original is like reading wikipedia summary after reading'

In [13]:
pre_data_tokens = pre_data.split(" ")
test_data_tokens = test_data.split(" ")

print('Unique Tokens: %d' % len(set(pre_data_tokens)))

Unique Tokens: 16277


In [14]:
pre_data_tokens[0:10]

['scare',
 'me',
 'final',
 'destination',
 'and',
 'did',
 'not',
 'scare',
 'me',
 'but']

In [15]:
length = 3 # for trigrams
sequences = list()
for i in range(length, len(pre_data_tokens)):
	# select sequence of tokens
	seq = pre_data_tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
 
print(sequences[:2]) 
print('Total Sequences: %d' % len(sequences))

['scare me final', 'me final destination']
Total Sequences: 250153


In [16]:
length = 3 # for trigrams
sequences_test = list()
for i in range(length, len(test_data_tokens)):
	# select sequence of tokens
	seq = test_data_tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences_test.append(line)
 
print(sequences_test[:2]) 
print('Total Sequences: %d' % len(sequences_test))

['a takura yamashita', 'takura yamashita koji']
Total Sequences: 36572


In [17]:
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

In [18]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences_int = tokenizer.texts_to_sequences(sequences)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [19]:
# integer encode sequences of words
tokenizer.fit_on_texts(sequences_test)
sequences_int_test = tokenizer.texts_to_sequences(sequences_test)

In [20]:
sequences_int[0]

[2405, 69, 469]

In [21]:
vocab_size

16277

In [22]:
sequences_int[0:2]

[[2405, 69, 469], [69, 469, 3767]]

In [23]:
length = 3
print(len(sequences_int))

250153


In [24]:
sequences_array = np.array(sequences_int)
sequences_array

array([[2405,   69,  469],
       [  69,  469, 3767],
       [ 469, 3767,    2],
       ...,
       [1582,    5,    3],
       [   5,    3,  805],
       [   3,  805,  435]])

In [25]:
sequences_array_test = np.array(sequences_int_test)
sequences_array_test

array([[    3, 17511,  1341],
       [17511,  1341, 16513],
       [ 1341, 16513,  9649],
       ...,
       [   20,   767,    18],
       [  767,    18,    14],
       [   18,    14,     3]])

In [26]:
len(sequences_array)

250153

In [27]:
c=1
h=0
while(c!=0):
    limit = len(sequences_array)
    d =[]
    c=0
    for i in range(0 , limit):
      # print(sequences_array[i])
      l=len(sequences_array[i])
      if(l!=length):
        print(sequences_array[i])
        sequences_array = np.delete(sequences_array, i, 0)
            
        c=1
        print("error at ", i)
        print("Deleting :_________________________ ") 
        h = h+1
        break  

print("Total deletion: ", h)  

Total deletion:  0


In [28]:
c=1
h=0
while(c!=0):
    limit = len(sequences_array_test)
    d =[]
    c=0
    for i in range(0 , limit):
      # print(sequences_array[i])
      l=len(sequences_array_test[i])
      if(l!=length):
        print(sequences_array_test[i])
        sequences_array_test = np.delete(sequences_array_test, i, 0)
            
        c=1
        print("error at ", i)
        print("Deleting :_________________________ ") 
        h = h+1
        break  

print("Total deletion: ", h) 

Total deletion:  0


In [29]:
sequences_final = np.array(sequences_array.tolist())
sequences_final

array([[2405,   69,  469],
       [  69,  469, 3767],
       [ 469, 3767,    2],
       ...,
       [1582,    5,    3],
       [   5,    3,  805],
       [   3,  805,  435]])

In [30]:
sequences_final_test = np.array(sequences_array_test.tolist())
sequences_final_test

array([[    3, 17511,  1341],
       [17511,  1341, 16513],
       [ 1341, 16513,  9649],
       ...,
       [   20,   767,    18],
       [  767,    18,    14],
       [   18,    14,     3]])

In [31]:
sequences_final.shape

(250153, 3)

In [32]:
# separate into input and output
X, y = sequences_final[:,:-1], sequences_final[:,-1]
seq_length = X.shape[1]

In [33]:
print(X[0], "->", y[0])

[2405   69] -> 469


In [34]:
X_test, y_test = sequences_final_test[:,:-1], sequences_final_test[:,-1]

In [35]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [36]:
X_ttr, X_tval, y_ttr, y_tval = train_test_split(X_test, y_test, test_size = 0.1, random_state = 42)

In [39]:
model = Sequential()
model.add(Embedding(vocab_size, 25, input_length=seq_length))
# model.add(GRU(150, return_sequences=True))
model.add(GRU(150, return_sequences=True))
model.add(GRU(100))
model.add(Dropout(0.4))
# model.add(Dense(150, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())


# es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X_tr, y_tr, batch_size=512, epochs=200)#, callbacks=[es_callback])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2, 25)             406925    
_________________________________________________________________
gru_3 (GRU)                  (None, 2, 150)            79650     
_________________________________________________________________
gru_4 (GRU)                  (None, 100)               75600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16277)             1643977   
Total params: 2,206,152
Trainable params: 2,206,152
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
391/391 [==============================] - 9s 16ms/step - loss: 7.7431 - accuracy: 0.0550
Epo

In [ ]:
# saving the model
# serialize to JSON
json_file = model.to_json()
with open("Model-JSON-N/Therapy.json", "w") as file:
   file.write(json_file)

# serialize weights to HDF5
model.save_weights("Model-JSON-N/Therapy-weights.h5")

In [ ]:
"""
from keras.models import model_from_json
# load json and create model
file = open(json_file, 'r')
model_json = file.read()
file.close()
loaded_model = model_from_json(model_json)
# load weights
loaded_model.load_weights(h5_file)
"""
print()

In [42]:
print(X_ttr[0], "->", y_ttr[0])

[2640  737] -> 4


In [44]:
results = model.evaluate(X_val, y_val)
print('Test accuracy: {0:.2f}%'.format(results[1]*100))

1564/1564 [==============================] - 6s 4ms/step - loss: 6.9517 - accuracy: 0.1417
Test accuracy: 14.17%
